Keybert for sample subset of each cluster

In [ ]:
* attach DOI to each cluster
* attach topic1 and topic2 to each cluster

In [ ]:
5

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()

def get_kw(text, kw_model=kw_model):
    kws = kw_model.extract_keywords(text, 
                          keyphrase_ngram_range=(1, 2), 
                          stop_words='english',
                          use_mmr=True, 
                          diversity=0.2)
    
    kws.sort(key=lambda x: x[1], reverse=True)
    return kws

In [ ]:
{cluster: [cluster],
doi: [dois],
topic: topicname,

}
for cluster in clusters:


In [ ]:
## Ingest the processed docs into the HierarchicalTagger pipeline

# Load topic labels from precomputed datast
# If you ran Engines on your own data, change the path to where you stored the output.
PRECOMPUTED_ITEM_TOPICS = os.path.join(ROOT_DIR, "./examples/data/amazon_products_precomputed.json")
with open(PRECOMPUTED_ITEM_TOPICS, "r") as f:
    topics = json.load(f)

# Confirm we have topics for all 3000 documents
len(topics)

JSON(topics)



# Inspect results for an item. We are interested in the `topics` key
topics["96d96237978ba26bbc6baa437372527a"]

!pwd
!pip install -r ../requirements.txt

import sys
sys.path
sys.path.insert(0, "/home/ec2-user/SageMaker/primer-hierarchical-tagger/") 

!pip install sentence-transformers

!pip install treelib scikit-network


# Import module and create a HierarchicalTagger instance
from hierarchical_tagger.hierarchical_tagger import HierarchicalTagger
hierarchical_tagger = HierarchicalTagger()

# Rework the document topic representations into a dictionary mapping document_id: List[topic labels as str]
document_topics = {document_id: topics_entry['topics'] for document_id, topics_entry in topics.items()}

JSON(document_topics)

# Send the document ids and their corresponding topic labels for ingest
hierarchical_tagger.ingest(document_terms=document_topics)

# The previous step is the most computationally demanding.
# To avoid having to repeat this, we can save our HierarchicalTagger instance to a json file, using the .to_json() helper method.
# As this file will also be the input data to our web app, so let's save it in `webapp/data/`
SERIALIZED_INSTANCE_PATH = os.path.join(ROOT_DIR, "./webapp/data/amazon_products.json")
with open(SERIALIZED_INSTANCE_PATH, "w") as f:
    f.write(hierarchical_tagger.to_json())

# This is how we can reload a HierarchicalTagger instance from file
with open(SERIALIZED_INSTANCE_PATH, "r") as f:
    reloaded_serialized =  json.load(f)
hierarchical_tagger = HierarchicalTagger.from_dict(reloaded_serialized)

## Build the topic tree and tag the documents

#### Hierarchical topic tree

The `.fit_tag_tree()` method populates the `.tree` attribute with a [treelib](https://treelib.readthedocs.io/en/latest/) object representing the extracted term tree. This can be manipulated and explored with all the treelib methods, for example `.show()` to print out a text representation of the tree.

# Fit the tree
hierarchical_tagger.fit_tag_tree()
hierarchical_tagger.tree.show()

# The final step is tagging the original documents based on the hierarchy we found in the tree
hierarchical_tagger.tag_documents()

#### Document tags

Inspect the `document_tags` attribute: a dictionary mapping document `id` to a list of tuples of the form `(term, score, node_id)` sorted by descending score. `score` measures how close in meaning the term is to the document. We would expect higher level abstractions to have lower scores.
`node_id` loosely indicates how high the node is in the tree: it's not a perfect measure, but more abstract terms will generally have higher `node_id`s.

# Let's see how the tags assinged to the Hover Board example we saw above
hierarchical_tagger.document_tags["96d96237978ba26bbc6baa437372527a"]

## Tuning parameters and using the web app

We expose several tuning parameters that the investigator can tweak to guide the extraction of the term tree and the logic applied when tagging the documents. You can find detailed documentation [here](https://github.com/PrimerAI/hierarchical-tagger/blob/main/hierarchical_tagger/hierarchical_tagger.py#L33). 

This repository also includes a simple web app to facilitate this iterative exploration. In a terminal, just run activate this tutorial's virtual enviroment and run the app like this:

```
$ workon ht-repo # Or alternative command to activate your virtual environment
$ streamlit run webapp/app.py
```

You can find more detailed instructions in our [companion tutorial](TODO_ADDLINK).


## Congratulations!

We hope you enjoyed this tutorial! Can you think of any other data you could run this on?




# Let's see how the tags assinged to the Hover Board example we saw above
hierarchical_tagger.document_tags["96d96237978ba26bbc6baa437372527a"]